# 텍스트 요약



###
DocumentEncoder

입력 문서(문장)를 임베딩 후 양방향 LSTM으로 인코딩합니다.
LSTM의 출력(각 토큰별 특징)과 마지막 hidden state를 hidden_size로 변환해 반환합니다.
즉, 문서 전체를 벡터 시퀀스로 변환해 요약에 필요한 정보를 추출합니다.


SummaryDecoder

인코더의 출력과 이전 hidden state를 받아 어텐션(attention)으로 컨텍스트 벡터를 계산합니다.
임베딩된 입력 토큰과 컨텍스트 벡터를 결합해 LSTM에 입력합니다.
LSTM의 출력을 통해 다음 요약 토큰을 예측합니다.
어텐션을 사용해 인코더의 중요한 부분을 동적으로 참고하며 요약을 생성합니다.
즉, DocumentEncoder는 문서의 의미를 벡터로 추출하고, SummaryDecoder는 그 정보를 바탕으로 어텐션을 활용해 요약을 생성하는 역할입니다

In [3]:
# 필요한 라이브러리 설치
!pip install torch


In [5]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import re
from collections import Counter

# 샘플 텍스트 요약 데이터 (문서-요약 쌍)
documents = [
    """
    인공지능은 현재 우리 생활의 많은 부분에 영향을 미치고 있습니다.
    스마트폰의 음성 인식, 검색 엔진의 결과 추천, 온라인 쇼핑의 상품 추천 등이
    모두 AI 기술의 응용 사례입니다. 특히 딥러닝의 발전으로 이미지 인식,
    자연어 처리, 게임 AI 등 다양한 분야에서 혁신적인 성과를 보이고 있습니다.
    앞으로 AI는 의료, 교육, 교통 등 더 많은 분야에서 활용될 것으로 예상됩니다.
    """,
    """
    기후 변화는 지구 온난화로 인해 발생하는 현상으로, 전 세계적으로 심각한
    문제가 되고 있습니다. 이산화탄소 배출량 증가가 주요 원인이며, 이로 인해
    극지방의 빙하가 녹고 해수면이 상승하고 있습니다. 또한 극한 기후 현상이
    빈번해지면서 가뭄, 홍수, 태풍 등의 자연재해가 증가하고 있습니다.
    이를 해결하기 위해서는 재생에너지 사용 확대와 탄소 배출 감소가 필요합니다.
    """,
    """
    코로나19 팬데믹은 전 세계 사람들의 생활 방식을 크게 바꾸어 놓았습니다.
    재택근무와 온라인 수업이 일반화되면서 디지털 기술의 중요성이 더욱 부각되었습니다.
    또한 언택트 문화가 확산되면서 온라인 쇼핑, 배달 서비스, 화상 회의 등이
    급속히 성장했습니다. 백신 개발과 보급을 통해 상황이 개선되고 있지만,
    팬데믹이 가져온 변화들은 앞으로도 지속될 것으로 보입니다.
    """,
    """
    전자상거래 시장은 코로나19 이후 급속한 성장을 보이고 있습니다.
    온라인 쇼핑의 편의성과 다양한 상품 선택권으로 인해 소비자들의
    선호도가 높아지고 있습니다. 특히 모바일 쇼핑의 증가가 두드러지며,
    소셜 커머스와 라이브 쇼핑 등 새로운 형태의 판매 방식도 등장하고 있습니다.
    기업들은 개인화된 추천 시스템과 빠른 배송 서비스를 통해 경쟁력을 높이고 있습니다.
    """,
    """
    환경 보호는 현재 가장 중요한 글로벌 이슈 중 하나입니다.
    플라스틱 사용 줄이기, 재활용, 에너지 절약 등 개인 차원의 노력이 필요합니다.
    정부와 기업들도 친환경 정책과 기술 개발에 투자하고 있습니다.
    지속가능한 발전을 위해서는 모든 사람의 참여가 중요합니다.
    미래 세대를 위해 지금부터 행동해야 합니다.
    """
]

summaries = [
    "AI는 우리 생활 곳곳에 활용되고 있으며 딥러닝 발전으로 다양한 분야에서 혁신을 이루고 있다",
    "기후 변화는 이산화탄소 증가로 인한 지구 온난화 현상으로 재생에너지 확대가 필요하다",
    "코로나19로 인해 재택근무 온라인 수업 등 디지털 기술 기반의 언택트 문화가 확산되었다",
    "전자상거래는 코로나19 이후 급성장하며 모바일 쇼핑과 새로운 판매 방식이 등장하고 있다",
    "환경 보호를 위해 개인과 정부 기업의 노력이 필요하며 지속가능한 발전을 위한 참여가 중요하다"
]

# 특수 토큰 정의
SOS_token = 0
EOS_token = 1
PAD_token = 2

# 언어 클래스 (번역 예제와 동일)
class Language:
    def __init__(self, name):
        self.name = name
        self.word2index = {"SOS": 0, "EOS": 1, "PAD": 2}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "PAD"}
        self.n_words = 3

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# 텍스트 전처리
def preprocess_text(text):
    # 줄바꿈 제거 및 공백 정리
    text = re.sub(r'\s+', ' ', text.strip())
    # 구두점 제거
    text = re.sub(r'[.!?,:;()]', '', text)
    return text.lower()

# 언어 모델 구축
doc_lang = Language('document')
summary_lang = Language('summary')

# 전처리된 문서-요약 쌍
pairs = []
for doc, summ in zip(documents, summaries):
    doc_processed = preprocess_text(doc)
    summ_processed = preprocess_text(summ)
    pairs.append([doc_processed, summ_processed])
    doc_lang.addSentence(doc_processed)
    summary_lang.addSentence(summ_processed)

print(f"문서 어휘 크기: {doc_lang.n_words}")
print(f"요약 어휘 크기: {summary_lang.n_words}")

# 인코더 (문서 인코더) - 차원 통일
class DocumentEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2):
        super(DocumentEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers,
                           batch_first=True, bidirectional=True)

        # bidirectional LSTM 출력을 hidden_size로 맞춤
        self.fc_hidden = nn.Linear(hidden_size * 2, hidden_size)
        self.fc_output = nn.Linear(hidden_size * 2, hidden_size)

    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        outputs, (hidden, cell) = self.lstm(embedded)

        # 양방향 LSTM 출력을 hidden_size로 변환
        outputs = self.fc_output(outputs)  # [batch, seq_len, hidden_size]

        # 양방향 LSTM의 마지막 hidden state 결합
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        hidden = self.fc_hidden(hidden).unsqueeze(0)  # [1, batch, hidden_size]

        return outputs, hidden

# 어텐션 디코더 (요약 디코더) - 간단한 어텐션
class SummaryDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers=1, dropout_p=0.1):
        super(SummaryDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(hidden_size * 2, hidden_size, num_layers, batch_first=True)  # 임베딩 + 컨텍스트

        # 어텐션 메커니즘
        self.attention = nn.Linear(hidden_size * 2, hidden_size)
        self.attn_v = nn.Linear(hidden_size, 1)

        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input_token, hidden, cell, encoder_outputs):
        # 입력 토큰 차원 조정
        if input_token.dim() == 1:
            input_token = input_token.unsqueeze(1)
        elif input_token.dim() == 3:
            input_token = input_token.squeeze(1)

        embedded = self.embedding(input_token)  # [batch, 1, hidden_size]
        embedded = self.dropout(embedded)

        # hidden state 차원 조정
        if hidden.dim() == 3:
            hidden_for_attn = hidden.squeeze(0)  # [batch, hidden_size]
        else:
            hidden_for_attn = hidden

        # 어텐션 계산
        seq_len = encoder_outputs.size(1)
        hidden_repeated = hidden_for_attn.unsqueeze(1).repeat(1, seq_len, 1)  # [batch, seq_len, hidden_size]

        # 어텐션 에너지 계산 (단순화)
        energy = self.attention(torch.cat((hidden_repeated, encoder_outputs), dim=2))  # [batch, seq_len, hidden_size]
        attention_weights = F.softmax(self.attn_v(energy).squeeze(2), dim=1)  # [batch, seq_len]

        # 컨텍스트 벡터 계산
        context = torch.sum(encoder_outputs * attention_weights.unsqueeze(2), dim=1)  # [batch, hidden_size]
        context = context.unsqueeze(1)  # [batch, 1, hidden_size]

        # 임베딩과 컨텍스트 결합
        lstm_input = torch.cat([embedded, context], dim=2)  # [batch, 1, hidden_size * 2]

        # LSTM forward
        output, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))

        # 최종 출력
        output = F.log_softmax(self.out(output.squeeze(1)), dim=1)
        return output, hidden, cell, attention_weights

# 텐서 변환 함수
def indexesFromSentence(lang, sentence):
    return [lang.word2index.get(word, PAD_token) for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long)

# 데이터셋 클래스
class SummarizationDataset(Dataset):
    def __init__(self, pairs, doc_lang, summary_lang):
        self.pairs = pairs
        self.doc_lang = doc_lang
        self.summary_lang = summary_lang

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        pair = self.pairs[idx]
        doc_tensor = tensorFromSentence(self.doc_lang, pair[0])
        summary_tensor = tensorFromSentence(self.summary_lang, pair[1])
        return doc_tensor, summary_tensor

# 패딩 함수
def collate_fn_summary(batch):
    doc_batch, summary_batch = zip(*batch)

    # 최대 길이 계산
    max_doc_len = max([len(seq) for seq in doc_batch])
    max_summary_len = max([len(seq) for seq in summary_batch])

    # 패딩 적용
    padded_docs = []
    padded_summaries = []

    for doc, summary in zip(doc_batch, summary_batch):
        # 문서 패딩
        if len(doc) < max_doc_len:
            padded_doc = torch.cat([doc, torch.full((max_doc_len - len(doc),), PAD_token)])
        else:
            padded_doc = doc
        padded_docs.append(padded_doc)

        # 요약 패딩
        if len(summary) < max_summary_len:
            padded_summary = torch.cat([summary, torch.full((max_summary_len - len(summary),), PAD_token)])
        else:
            padded_summary = summary
        padded_summaries.append(padded_summary)

    return torch.stack(padded_docs), torch.stack(padded_summaries)

# 모델 초기화
hidden_size = 256
encoder = DocumentEncoder(doc_lang.n_words, hidden_size)
decoder = SummaryDecoder(hidden_size, summary_lang.n_words)

# 데이터 로더
dataset = SummarizationDataset(pairs, doc_lang, summary_lang)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn_summary)

# 훈련 함수
def train_summarization(doc_tensor, summary_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    doc_length = doc_tensor.size(1)
    summary_length = summary_tensor.size(1)

    # 인코더 forward
    encoder_outputs, encoder_hidden = encoder(doc_tensor)

    # 디코더 초기화
    decoder_input = torch.tensor([[SOS_token]], dtype=torch.long)
    decoder_hidden = encoder_hidden
    decoder_cell = torch.zeros_like(decoder_hidden)

    loss = 0

    # Teacher forcing 사용
    for di in range(summary_length - 1):
        decoder_output, decoder_hidden, decoder_cell, attention_weights = decoder(
            decoder_input, decoder_hidden, decoder_cell, encoder_outputs)

        loss += criterion(decoder_output, summary_tensor[:, di])
        decoder_input = summary_tensor[:, di].unsqueeze(0).unsqueeze(0)

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / (summary_length - 1)

# 훈련 설정
learning_rate = 0.001
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

# 훈련 실행
print("요약 모델 훈련 시작...")
for epoch in range(200):
    total_loss = 0
    for doc_tensor, summary_tensor in dataloader:
        loss = train_summarization(doc_tensor, summary_tensor, encoder, decoder,
                                 encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    if epoch % 40 == 0:
        print(f'Epoch {epoch}, Loss: {total_loss:.4f}')

# 요약 함수
def summarize(document, max_length=25):
    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        doc_processed = preprocess_text(document)
        doc_tensor = tensorFromSentence(doc_lang, doc_processed).unsqueeze(0)

        encoder_outputs, encoder_hidden = encoder(doc_tensor)

        decoder_input = torch.tensor([[SOS_token]], dtype=torch.long)
        decoder_hidden = encoder_hidden
        decoder_cell = torch.zeros_like(decoder_hidden)

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_cell, attention_weights = decoder(
                decoder_input, decoder_hidden, decoder_cell, encoder_outputs)

            topv, topi = decoder_output.topk(1)
            if topi.item() == EOS_token:
                break
            else:
                decoded_words.append(summary_lang.index2word[topi.item()])

            decoder_input = topi.detach().unsqueeze(0)

        return ' '.join(decoded_words)

# ROUGE 점수 계산 (간단 버전)
def simple_rouge_1(reference, hypothesis):
    ref_words = set(reference.split())
    hyp_words = set(hypothesis.split())

    if len(ref_words) == 0:
        return 0.0

    overlap = len(ref_words.intersection(hyp_words))
    precision = overlap / len(hyp_words) if len(hyp_words) > 0 else 0
    recall = overlap / len(ref_words)
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return f1

# 테스트
print("\n=== 텍스트 요약 결과 ===")
test_documents = [
    """
    스마트폰은 현대인의 필수품이 되었습니다.
    통화, 메시지 전송뿐만 아니라 인터넷 검색, 사진 촬영,
    게임, 음악 감상 등 다양한 기능을 제공합니다.
    특히 모바일 앱의 발달로 은행 업무, 쇼핑,
    교육 등의 활동도 스마트폰으로 가능해졌습니다.
    """,
    """
    건강한 식습관은 우리 몸에 매우 중요합니다.
    균형 잡힌 영양소 섭취와 규칙적인 식사가 필요합니다.
    과도한 당분과 지방 섭취는 피하고 신선한 채소와
    과일을 많이 먹어야 합니다. 충분한 수분 섭취도 잊지 말아야 합니다.
    """
]

for i, doc in enumerate(test_documents):
    summary = summarize(doc)
    print(f"문서 {i+1}:")
    print(f"원문: {doc.strip()}")
    print(f"요약: {summary}")
    print("-" * 50)

# 훈련 데이터에 대한 성능 확인
print("\n=== 훈련 데이터 요약 성능 ===")
total_rouge = 0
for i, (doc, ref_summary) in enumerate(zip(documents, summaries)):
    generated_summary = summarize(doc)
    rouge_score = simple_rouge_1(preprocess_text(ref_summary), generated_summary)
    total_rouge += rouge_score

    print(f"문서 {i+1} ROUGE-1 F1: {rouge_score:.3f}")
    print(f"참조: {ref_summary}")
    print(f"생성: {generated_summary}")
    print("-" * 30)

print(f"평균 ROUGE-1 F1: {total_rouge/len(documents):.3f}")

문서 어휘 크기: 195
요약 어휘 크기: 62
요약 모델 훈련 시작...
Epoch 0, Loss: 20.8162
Epoch 40, Loss: 0.0663
Epoch 80, Loss: 0.0228
Epoch 120, Loss: 0.0121
Epoch 160, Loss: 0.0076

=== 텍스트 요약 결과 ===
문서 1:
원문: 스마트폰은 현대인의 필수품이 되었습니다. 
    통화, 메시지 전송뿐만 아니라 인터넷 검색, 사진 촬영, 
    게임, 음악 감상 등 다양한 기능을 제공합니다.
    특히 모바일 앱의 발달로 은행 업무, 쇼핑, 
    교육 등의 활동도 스마트폰으로 가능해졌습니다.
요약: 전자상거래는 코로나19 이후 급성장하며 모바일 쇼핑과 새로운 판매 방식이 등장하고 있다 있다 있다 있다 있다 있다 있다 있다 있다 있다 필요하다 증가로 인한 지구 온난화
--------------------------------------------------
문서 2:
원문: 건강한 식습관은 우리 몸에 매우 중요합니다.
    균형 잡힌 영양소 섭취와 규칙적인 식사가 필요합니다.
    과도한 당분과 지방 섭취는 피하고 신선한 채소와 
    과일을 많이 먹어야 합니다. 충분한 수분 섭취도 잊지 말아야 합니다.
요약: 전자상거래는 코로나19 이후 급성장하며 모바일 쇼핑과 새로운 판매 방식이 등장하고 있다 있다 있다 있다 있다 있다 필요하다 증가로 인한 지구 온난화 현상으로 재생에너지 확대가 필요하다
--------------------------------------------------

=== 훈련 데이터 요약 성능 ===
문서 1 ROUGE-1 F1: 1.000
참조: AI는 우리 생활 곳곳에 활용되고 있으며 딥러닝 발전으로 다양한 분야에서 혁신을 이루고 있다
생성: ai는 우리 생활 곳곳에 활용되고 있으며 딥러닝 발전으로 다양한 분야에서 혁신을 이루고 있다 있다 있다 있다 있다 있다 있다 있다 있다 있다 있다 있다 있다
---------------